## Importing

In [1]:
# user configuration

# toggle to create models with/without outliers
remove_outliers_flag = True

# toggle convert scores (from letters to numbers)
convert_scores_flag = True

# toggle scaling
perform_scaling_flag = True

In [2]:
# import libraries

# maths
import scipy.stats as stats
import numpy as np
import pandas as pd
#from pandas.api.types import is_numeric_dtype

# visual
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# machine learning
from sklearn.linear_model import LinearRegression,Ridge,RidgeCV,Lasso,LassoCV,ElasticNet 
from sklearn import linear_model
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.linear_model import Ridge

# html
#from IPython.display import Image
#from IPython.core.display import HTML

# other
#import re
#import os

In [3]:
# import raw data
input_path = '../data/input/'

df_train = pd.read_csv(input_path + 'train.csv')
df_test = pd.read_csv(input_path + 'test.csv')

# import helper files
mid_path = '../data/mid/'

df_cols = pd.read_csv(mid_path + 'columns.csv',header=None)

## Functions

In [4]:
# convert columns with rating (letters) to enums (numbers)
def convert_scores(col,scole_type):
    
        col = col.str.lower()
        
        if scole_type == 1:
            score = {'na':0,'fa':1,'ta':2,'gd':3,'ex':4}
        elif scole_type == 2:        
            score = {'na':0,'po':1,'fa':2,'ta':3,'gd':4,'ex':5}
        elif scole_type == 3:
            score = {'na':0,'unf':1,'lwq':2,'rec':3,'blq':4,'alq':5,'glq':6}
        else:
            print(str(scole_type) + ' is invalid scole_type!')
            return col
            
        for key in score.keys():
                col = col.replace(key, score[key])
                
        return col

In [5]:
def create_scatterplot(title,data,x,y,hue=None,xlabel=None,ylabel=None):    
        
    plt.figure(figsize=(10,8))
    
    sns.set_style("whitegrid")
    palette = sns.color_palette("bright",n_colors=16)
      
    #sns.regplot(data,x=x,y=y)        
    sns.scatterplot(data=data,x=x,y=y,hue=hue,palette=palette)

    #plt.xlim(min_x,max_x)
    #plt.ylim(min_x,max_x)
                    
    plt.title(title)
    
    if xlabel == None:
        xlabel = x
    if ylabel == None:
        ylabel = y
    
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)

In [6]:
def create_boxplot(data,x,y):
    
    plt.figure(figsize=(20,8))
    
    g = sns.boxplot(data=data,x=x,y=y)
    #g.set(xlabel=xlabel)
    #g.set(xticklabels=xlabel)

In [7]:
lr = LinearRegression()

def create_lm_model(df,y,cols,cv=5):
    
    X = df.loc[:,cols]
    X = X.values

    model = lr.fit(X,y)
    print('coef: {} intercept: {}'.format(model.coef_,model.intercept_))

    cvs = cross_val_score(model,X,y,cv=5)    
    mean = np.mean(cvs)
    print('cvs:',cvs)
    print('mean:',mean)
    
    return model

## Inspect df_train

In [8]:
print(df_train.columns)

Index(['Id', 'PID', 'MS SubClass', 'MS Zoning', 'Lot Frontage', 'Lot Area',
       'Street', 'Alley', 'Lot Shape', 'Land Contour', 'Utilities',
       'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1',
       'Condition 2', 'Bldg Type', 'House Style', 'Overall Qual',
       'Overall Cond', 'Year Built', 'Year Remod/Add', 'Roof Style',
       'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type',
       'Mas Vnr Area', 'Exter Qual', 'Exter Cond', 'Foundation', 'Bsmt Qual',
       'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin SF 1',
       'BsmtFin Type 2', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF',
       'Heating', 'Heating QC', 'Central Air', 'Electrical', '1st Flr SF',
       '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Bsmt Full Bath',
       'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr',
       'Kitchen AbvGr', 'Kitchen Qual', 'TotRms AbvGrd', 'Functional',
       'Fireplaces', 'Fireplace Qu', 'Garage Type', 'Garage Yr Blt',
       'G

In [9]:
df_train.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


In [10]:
df_train_info = df_train.describe()
df_train_info

,Id,PID,MS SubClass,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,SalePrice
count,2051.000000,2.051000e+03,2051.000000,1721.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2029.000000,...,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000
mean,1474.033642,7.135900e+08,57.008776,69.055200,10065.208191,6.112140,5.562165,1971.708922,1984.190151,99.695909,...,93.833740,47.556802,22.571916,2.591419,16.511458,2.397855,51.574354,6.219893,2007.775719,181469.701609
std,843.980841,1.886918e+08,42.824223,23.260653,6742.488909,1.426271,1.104497,30.177889,21.036250,174.963129,...,128.549416,66.747241,59.845110,25.229615,57.374204,37.782570,573.393985,2.744736,1.312014,79258.659352
min,1.000000,5.263011e+08,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,12789.000000
25%,753.500000,5.284581e+08,20.000000,58.000000,7500.000000,5.000000,5.000000,1953.500000,1964.500000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000,129825.000000
50%,1486.000000,5.354532e+08,50.000000,68.000000,9430.000000,6.000000,5.000000,1974.000000,1993.000000,0.000000,...,0.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,162500.000000
75%,2198.000000,9.071801e+08,70.000000,80.000000,11513.500000,7.000000,6.000000,2001.000000,2004.000000,161.000000,...,168.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,2930.000000,9.241520e+08,190.000000,313.000000,159000.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,...,1424.000000,547.000000,432.000000,508.000000,490.000000,800.000000,17000.000000,12.000000,2010.000000,611657.000000


In [11]:
df_train_list = df_train.dtypes
print(df_train_list)

Id                  int64
PID                 int64
MS SubClass         int64
MS Zoning          object
Lot Frontage      float64
Lot Area            int64
Street             object
Alley              object
Lot Shape          object
Land Contour       object
Utilities          object
Lot Config         object
Land Slope         object
Neighborhood       object
Condition 1        object
Condition 2        object
Bldg Type          object
House Style        object
Overall Qual        int64
Overall Cond        int64
Year Built          int64
Year Remod/Add      int64
Roof Style         object
Roof Matl          object
Exterior 1st       object
Exterior 2nd       object
Mas Vnr Type       object
Mas Vnr Area      float64
Exter Qual         object
Exter Cond         object
                   ...   
Half Bath           int64
Bedroom AbvGr       int64
Kitchen AbvGr       int64
Kitchen Qual       object
TotRms AbvGrd       int64
Functional         object
Fireplaces          int64
Fireplace Qu

In [12]:
# Check for nulls in columns
null_cols = df_train.isnull().sum()
mask_null = null_cols > 0
null_cols[mask_null].sort_values(ascending=False)

Pool QC           2042
Misc Feature      1986
Alley             1911
Fence             1651
Fireplace Qu      1000
Lot Frontage       330
Garage Yr Blt      114
Garage Cond        114
Garage Qual        114
Garage Finish      114
Garage Type        113
Bsmt Exposure       58
BsmtFin Type 2      56
Bsmt Cond           55
Bsmt Qual           55
BsmtFin Type 1      55
Mas Vnr Type        22
Mas Vnr Area        22
Bsmt Half Bath       2
Bsmt Full Bath       2
Garage Cars          1
BsmtFin SF 1         1
BsmtFin SF 2         1
Bsmt Unf SF          1
Garage Area          1
Total Bsmt SF        1
dtype: int64

In [13]:
# Check for nulls in rows
null_rows = df_train.isnull().sum(axis=1)
mask_null = null_rows > 0
null_rows[mask_null].sort_values(ascending=False)

114     16
580     15
437     15
1327    15
2010    15
1594    15
1428    15
1098    14
700     14
256     14
276     12
1007    11
1049    11
737     11
514     11
1211    11
616     11
1372    11
334     11
1125    11
1464    10
1686    10
964     10
954     10
942     10
924     10
901     10
878     10
824     10
1799    10
        ..
1961     3
1844     3
1471     3
159      3
1989     2
761      2
657      2
1225     2
1246     2
149      2
424      2
1134     2
1635     2
29       2
1422     2
1679     2
1707     2
1373     2
1002     2
2032     2
107      2
1652     2
1130     2
110      2
1133     2
1405     2
1756     2
1759     2
52       1
974      1
Length: 2051, dtype: int64

In [14]:
df_train_info = df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 81 columns):
Id                 2051 non-null int64
PID                2051 non-null int64
MS SubClass        2051 non-null int64
MS Zoning          2051 non-null object
Lot Frontage       1721 non-null float64
Lot Area           2051 non-null int64
Street             2051 non-null object
Alley              140 non-null object
Lot Shape          2051 non-null object
Land Contour       2051 non-null object
Utilities          2051 non-null object
Lot Config         2051 non-null object
Land Slope         2051 non-null object
Neighborhood       2051 non-null object
Condition 1        2051 non-null object
Condition 2        2051 non-null object
Bldg Type          2051 non-null object
House Style        2051 non-null object
Overall Qual       2051 non-null int64
Overall Cond       2051 non-null int64
Year Built         2051 non-null int64
Year Remod/Add     2051 non-null int64
Roof Style         20

In [15]:
# find columns with the most empty cells
df_train.count(axis=0).sort_values(ascending=False)

SalePrice          2051
Exter Qual         2051
Year Built         2051
Year Remod/Add     2051
Roof Style         2051
Roof Matl          2051
Exterior 1st       2051
Exterior 2nd       2051
Exter Cond         2051
Low Qual Fin SF    2051
Foundation         2051
Sale Type          2051
Heating QC         2051
Central Air        2051
Electrical         2051
1st Flr SF         2051
Overall Cond       2051
Overall Qual       2051
House Style        2051
Bldg Type          2051
Condition 2        2051
Condition 1        2051
Neighborhood       2051
Land Slope         2051
Lot Config         2051
Utilities          2051
Land Contour       2051
Lot Shape          2051
Street             2051
Lot Area           2051
                   ... 
Kitchen AbvGr      2051
Full Bath          2051
Half Bath          2051
Bedroom AbvGr      2051
Garage Cars        2050
Total Bsmt SF      2050
Bsmt Unf SF        2050
BsmtFin SF 2       2050
Garage Area        2050
BsmtFin SF 1       2050
Bsmt Half Bath  

## Clean df_train

In [16]:
# rename columns
df_train.columns = df_cols[1]

In [17]:
if remove_outliers_flag == True:
    
    print('before:',len(df_train))
    
    mask_no_outliers = df_train['ground_total_area'] > 4000
    df_train = df_train[~mask_no_outliers]
    
    print('after:',len(df_train))

before: 2051
after: 2049


In [18]:
# columns with nan
cols_nan = ['pool_qual','bsmt_qual','fireplace_score','garage_qual','garage_cond','bsmt_type1_score','bsmt_type2_score']

df_train = df_train[cols_nan].fillna(value=0,inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


## Inspect df_test

## Clean df_test 

## Output Clean Data

In [19]:
df_train['garage_cond'].unique()

TypeError: 'NoneType' object is not subscriptable

In [ ]:
# output to csv

mid_path = '../data/mid/'

#df_corr.to_csv(mid_path + 'df_corr.csv')
df_train.to_csv(mid_path + 'df_train_clean.csv')

## Prepare Data 

In [ ]:
cols_1 = ['pool_qual']
cols_2 = ['exterior_qual','exterior_cond','bsmt_qual','bsmt_cond','heating_cond','kitchen_qual','fireplace_score','garage_qual','garage_cond']
cols_3 = ['bsmt_type1_score','bsmt_type2_score']
cols_4 = ['house_qual','house_cond']

comb_cols = cols_1 + cols_2 +  cols_3 + cols_4

In [ ]:
# convert columns with rating (letters) to enums (numbers)

if convert_scores_flag == True:  

    for col in cols_1:    
        # df_train[col + '_2'] = convert_scores(df_train[col],scole_type=1) # test first before overwrite
        df_train[col] = convert_scores(df_train[col],scole_type=1) # overwrite  

    for col in cols_2:    
        # df_train[col + '_2'] = convert_scores(df_train[col],scole_type=2) # test first before overwrite
        df_train[col] = convert_scores(df_train[col],scole_type=2) # overwrite    

    for col in cols_3:    
        # df_train[col + '_2'] = convert_scores(df_train[col],scole_type=2) # test first before overwrite
        df_train[col] = convert_scores(df_train[col],scole_type=3) # overwrite

In [ ]:
if perform_scaling_flag == True:

    # create X (feature matrix) and y (target)  
    X = df_train.loc[:,comb_cols]
    X = X.values
    y = df_train['sale_price'].values
    #print(X[:5])    

    # perform scaling on X
    ss = StandardScaler()
    ss.fit(X)
    X2 = ss.fit_transform(X)
    
    #print(X2[:5])
    #print(X2[:,12].min(),X2[:,12].mean(),X2[:,12].max())
    
    # errors columns: 0 3 4 7 8 9 10 11
    test = 13
    sns.distplot(X[:,test])
    sns.distplot(X2[:,test]) 
    
    # for further checks
    #df_comb_cols = pd.DataFrame(comb_cols)
    #df_comb_cols.to_csv('df_comb_cols.csv')   

In [ ]:
#l_alphas = np.arange(0.001, 0.15, 0.0025)
#lasso_model = LassoCV(alphas=l_alphas, cv=5)
#lasso_model = lasso_model.fit(X2, y)

In [ ]:
#model = lr.fit(X2,y)
#print('coef: {} intercept: {}'.format(model.coef_,model.intercept_))

#cvs = cross_val_score(model,X,y,cv=5)    
#mean = np.mean(cvs)
#print('cvs:',cvs)
#print('mean:',mean)

## EDA

In [ ]:
# df_train heatmap
#fig, ax = plt.subplots(figsize=(20, 10))
#sns.heatmap(df_train.corr())

In [ ]:
# df_train heatmap (staircase)
corr = df_train.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True

fig, ax = plt.subplots(figsize=(20, 10))
sns.heatmap(corr, mask=mask, vmax=.3, square=True,cmap="coolwarm_r")    

#with sns.axes_style("whitegrid"):
    #ax = sns.heatmap(corr, mask=mask, vmax=.3, square=True,cmap="RdBu")

In [ ]:
df_train.head()

In [ ]:
# inspect correlation matrix
df_corr = df_train.corr()
df_corr['sale_price'].sort_values(ascending=False)

In [ ]:
data = df_train
x = 'house_qual'
y = 'sale_price'
hue = 'lot_subclass'

create_scatterplot('',data,x,y,hue) 

In [ ]:
data = df_train
x = 'year_built'
y = 'sale_price'

create_scatterplot('',data,x,y)

In [ ]:
data = df_train
x = 'year_remodel'
y = 'sale_price'

create_scatterplot('',data,x,y)

In [ ]:
data = df_train
x = 'ground_total_area'
y = 'sale_price'
hue = 'lot_subclass'

create_scatterplot('',data,x,y,hue)
print('found 2 outliers (bottom right)')

In [ ]:
# enable back later

#data = df_train
#x = 'interact_03' # df_train['house_qual'] * df_train['ground_total_area']
#y = 'sale_price'
#hue = 'lot_subclass'

#create_scatterplot('',data,x,y,hue)
#print('less variance compared to above plot')

In [ ]:
x = df_train['lot_subclass']
sns.distplot(x, bins=20)

In [ ]:
create_boxplot(df_train,x='lot_subclass',y='sale_price')

In [ ]:
create_boxplot(df_train,x='lot_zone',y='sale_price')

In [ ]:
create_boxplot(df_train,x='neighborhood',y='sale_price')

## Modelling

In [ ]:
# init
y_train = df_train['sale_price']

In [ ]:
# lr_model_01
cols = ['house_qual']
lr_model_01 = create_lm_model(df_train,y_train,cols,cv=5)

In [ ]:
# lr_model_01 (scaled)
#cols = ['house_qual']
#lr_model_01 = create_lm_model(X,y,cols,cv=5)

In [ ]:
# lr_model_02
cols = ['ground_total_area']
lr_model_02 = create_lm_model(df_train,y_train,cols,cv=5)

In [ ]:
# lr_model_03
cols = ['year_built']
lr_model_03 = create_lm_model(df_train,y_train,cols,cv=5)

In [ ]:
# lr_model_04
cols = ['year_remodel']
lr_model_04 = create_lm_model(df_train,y_train,cols,cv=5)

In [ ]:
# lr_model_05
cols = ['year_built','year_remodel']
lr_model_05 = create_lm_model(df_train,y_train,cols,cv=5)

In [ ]:
# lr_model_06
cols = ['house_qual','ground_total_area']
lr_model_06 = create_lm_model(df_train,y_train,cols,cv=5)

In [ ]:
# lr_model_07
cols = ['house_qual','year_remodel']
lr_model_07 = create_lm_model(df_train,y_train,cols,cv=5)

In [ ]:
# lr_model_08
cols = ['house_qual','ground_total_area','exterior_qual','kitchen_qual','floor_1_area']
lr_model_08 = create_lm_model(df_train,y_train,cols,cv=5)

In [ ]:
# lr_model_09
cols = ['pid']
lr_model_09 = create_lm_model(df_train,y_train,cols,cv=5)

In [ ]:
# lr_model_interact_01
df_train['interact_01'] = df_train['year_built'] * df_train['year_remodel']
cols = ['interact_01']
lr_model_interact_01 = create_lm_model(df_train,y_train,cols,cv=5)

In [ ]:
# lr_model_interact_02
df_train['interact_02'] = df_train['house_qual'] * df_train['year_remodel']
cols = ['interact_02']
lr_model_interact_02 = create_lm_model(df_train,y_train,cols,cv=5)

In [ ]:
# lr_model_interact_03
df_train['interact_03'] = df_train['house_qual'] * df_train['ground_total_area']
cols = ['interact_03']
lr_model_interact_03 = create_lm_model(df_train,y_train,cols,cv=5)